# SDS to website
Purpose: By processing a list of events from an Excel spreadsheet, look for existing seismic & infrasound data in an SDS archive, instrument correct those data, and make a webpage for each event 

Steps:
1. Create an event catalog with instrument-corrected data

TO DO: 
* add support for AM network, and any data from networks run by other operators and available on IRIS
* add well data from YYYYMMDD_final.pkl files

Do we want to turn pkl files into SDS?

SCAFFOLD:
* add a detector for sonic booms after rocket launch for landers
* add any nearby IRIS seismic and infrasound data

# 1. Do full workflow for all events
* Load and correct seismic and infrasound data from FL network and XA and whatever the network was for PASSCAL equipment

In [1]:
import os
import glob
import sys
import header
paths = header.setup_environment()
paths['SDS_TOP'] = '/data/SDS'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import obspy
sys.path.append(os.path.join(paths['Developer'], 'SoufriereHillsVolcano', 'lib'))
import SDS
import Spectrograms
import USF_instrument_responses
import libWellData as LLE
from obspy.clients.filesystem.sds import Client
from obspy.signal.cross_correlation import correlate, xcorr_max
from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import matplotlib.dates as dates


##############################################################
# written for well data

def dataframe2stream(df):
    stream = obspy.Stream()
    for column in df.columns:
        if column=='datetime':
            continue
        # Create Trace for each channel
        trace = obspy.Trace()
        trace.data = df[column].values
        if column=='AirPressureShallow':
            trace.id = '6S.Baro..HDD'
        elif column=='AirPressureDeep':
            trace.id = '6I.Baro..HDD'
        else:
            name = column.ljust(12, '_')
            if column in watercolumnsShallow:
                well = '6S'
                chan = 'WLM'
            elif column in watercolumnsIntermediate:
                well = '6I'
                chan = 'WLM'
            elif column in aircolumns:
                if column[-1]=='0':
                    well = '6S'
                else:
                    well = '6I'
                chan = 'HDD'
            trace.id = '.'.join((well, column[0:5], column[5:7], chan))
        if trace.stats.channel=='WLM':
            trace.stats['units'] = 'mh20'
        elif trace.stats.channel=='HDD':
            trace.stats['units'] = 'Pa'

        mintime = df['datetime'].min()
        try:
            trace.stats.starttime = obspy.UTCDateTime(mintime.timestamp())  # Set the start time
        except:
            print(mintime, df.loc[0,'datetime'])
            trace.stats.starttime = obspy.UTCDateTime(df.iloc[0]['datetime'].timestamp())
        trace.stats.delta = df['datetime'].diff().median().total_seconds()  # Sampling rate (1 Hz in this example)
        
        stream += trace  # Add the trace to the stream
    print(stream)
    return stream


def detectEvent(st):
    st2 = st.copy().filter('bandpass', freqmin=5, freqmax=16)
    N = len(st)
    #trig = coincidence_trigger("recstalta", 3.5, 1, st, 3, sta=2, lta=40)
    num_triggers_needed = max(( int(N/2), 2)) # half the channels or 2, whichever is greater
    trig = coincidence_trigger("recstalta", 4, 1, st2, num_triggers_needed, sta=5, lta=100, max_trigger_length=180, delete_long_trigger=True, details=True)
    best_trig = {}
    best_product = 0

    for this_trig in trig:

        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    return best_trig

def intersect(df, cols):
    common_cols = list(set(df.columns) & set(cols))
    return common_cols

#############################################################################
# from 25_SDS_to_eventfiles - and just used for seismic and infrasound now

def sds2eventStream(launchtime, sdsclient, thisSDSobj, pretrig=3600, posttrig=3600):  
      
    startt = obspy.UTCDateTime(launchtime) - pretrig
    endt = obspy.UTCDateTime(launchtime) + posttrig
    st = try_different_waveform_loading_methods(sdsclient, thisSDSobj, startt, endt)
    return st

def try_different_waveform_loading_methods(sdsclient, thisSDSobj, startt, endt):

    # ObsPy SDS archive reader
    st3 = sdsclient.get_waveforms("*", "*", "*", "[HDCES]*", startt, endt)

    # My SDS class that wraps ObsPy SDS reader
    thisSDSobj.read(startt, endt, speed=1)
    st4 = thisSDSobj.stream

    combine_streams(st3, st4)    
    
    return st3
    
def combine_streams(stB, stA):
    appended = False
    for trA in stA:
        found = False
        for trB in stB:
            if trA.stats.station == trB.stats.station and trA.stats.location == trB.stats.location and trA.stats.channel == trB.stats.channel:
                if trA.stats.network == '':
                    trA.stats.network = trB.stats.network
                if trA.stats.starttime >= trB.stats.starttime and trA.stats.endtime <= trB.stats.endtime:
                    found = True
                    break
        if not found:
            stB.append(trA)
            appended = True
    if appended:
        stB.merge(method=0, fill_value=0)


def clean(st, taperseconds):
    for tr in st:
        remove_single_sample_spikes(tr)
        try:
            tr.detrend('linear')
        except:
            #mask = np.isnan(data)
            #tr.data = np.ma.masked_array(data, mask)  # Apply mask to the data
            try:
                tr.data = tr.data - np.nanmedian(tr.data)
                if np.ma.is_masked(tr.data):
                    # Replace masked values with 0
                    tr.data = tr.data.filled(0)
            except:
                st.remove(tr)
                continue

        # taper
        trace_seconds = tr.stats.delta * tr.stats.npts
        tr.taper(max_percentage=taperseconds/trace_seconds)

        try:
            tr.filter('highpass', freq=0.01, corners=2) 
        except:
            st.remove(tr)
            continue

        tr.trim(starttime=tr.stats.starttime+taperseconds, endtime=tr.stats.endtime-taperseconds)
        
def apply_calibration_correction(st):
    # calibration correction

    for tr in st:
        if 'countsPerUnit' in tr.stats:
            continue
        else:
            tr.stats['countsPerUnit'] = 1
            if not 'units' in tr.stats:
                tr.stats['units'] = 'Counts'
            if tr.stats.station[0].isnumeric(): # well data
                if len(tr.stats.network)==0:
                    tr.stats.network = '6'
                if tr.stats.channel[2] == 'D':
                    tr.stats.countsPerUnit = 1/LLE.psi2inches(1) # counts (psi) per inch
                    tr.stats.units = 'inches'
                elif tr.stats.channel[2] == 'H':
                    tr.stats.countsPerUnit = 1/6894.76 # counts (psi) per Pa
                    tr.stats.units = 'Pa'
            elif tr.stats.channel[1]=='D':
                tr.stats.countsPerUnit = 720 # counts/Pa on 1 V FS setting
                if tr.id[:-1] == 'FL.BCHH3.10.HD':
                    if tr.stats.starttime < obspy.UTCDateTime(2022,5,26): # Chaparral M25. I had it set to 1 V FS. Should have used 40 V FS. 
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 8e5 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 720 # counts/Pa 
                    else: # Chaparral switched to 40 V FS
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 2e4 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 18 # counts/Pa 
                tr.stats.units = 'Pa'

            elif tr.stats.channel[1]=='H':
                tr.stats.countsPerUnit = 3e2 # counts/(um/s)
                tr.stats.units = 'um/s'
            tr.data = tr.data/tr.stats.countsPerUnit
    
def maxamp(tr):
    return np.max(np.abs(tr.data))

def remove_spikes(st):
    SEISMIC_MAX = 0.1 # m/s
    INFRASOUND_MAX = 3000 # Pa
    FEET_MAX = 21 # feet
    #SEISMIC_MIN = 1e-9
    #INFRASOUND_MIN = 0.01
    
    for tr in st:
        ma = maxamp(tr)

        #if tr.stats.units == 'm/s':
        if tr.stats.channel[1] == 'H':
            tr.data[tr.data > SEISMIC_MAX] = np.nan
            tr.data[tr.data < -1 * SEISMIC_MAX] = np.nan             
        elif tr.stats.channel[1] == 'D':
            tr.data[tr.data > INFRASOUND_MAX] = np.nan
            tr.data[tr.data < -1 * INFRASOUND_MAX] = np.nan   
        '''
        elif tr.stats.units == 'feet': # for when we had well data in feet in SDS
            tr.data[tr.data > FEET_MAX] = np.nan
            tr.data[tr.data < -1 * FEET_MAX] = np.nan   
        '''           



'''
def add_snr(st, assoctime, threshold=1.5):
    nstime = max([st[0].stats.starttime, assoctime-240])
    netime = min([st[0].stats.endtime, assoctime-60])
    sstime = assoctime
    setime = min([st[0].stats.endtime, assoctime+120])    
    for tr in st:
        tr_noise = tr.copy().trim(starttime=nstime, endtime=netime)
        tr_signal = tr.copy().trim(starttime=sstime, endtime=setime)
        tr.stats['noise'] = np.nanmedian(np.abs(tr_noise.data))
        tr.stats['signal'] = np.nanmedian(np.abs(tr_signal.data))
        tr.stats['snr'] = tr.stats['signal']/tr.stats['noise']
        '''
'''
def group_streams_for_plotting(st):
    groups = {}
    stationsWELL = ['6S', '6I']
    for station in stationsWELL:
        stationStream = st.select(network=station)
        #stationIDS = list(set([tr.id for tr in stationStream]))
        groups[station] = stationStream
    streamSA = st.select(network='FL')
    stationsSA = list(set([tr.stats.station for tr in streamSA]))
    for station in stationsSA:
        stationStream = streamSA.select(station=station)
        #stationIDS = list(set([tr.id for tr in stationStream]))
        groups[station] = stationStream
    #print(groups)
    return groups  '''
def remove_single_sample_spikes(trace):
    # Parameters
    threshold = 10  # Threshold for identifying large differences (spikes)
    trace_std = np.nanstd(trace.data)

    # Step 1: Calculate the absolute differences between consecutive points
    diff_prev = np.abs(np.diff(trace.data))  # Difference with the previous point
    diff_next = np.abs(np.diff(trace.data[1:], append=trace.data[-1]))  # Difference with the next point

    # Step 2: Identify spikes where the difference is larger than the threshold
    spikes = (diff_prev > trace_std * threshold) & (diff_next > trace_std * threshold)
    spikes = np.append(spikes, np.array(False))


    # Step 3: Replace spikes with the average of previous and next points
    # Use boolean indexing to replace the spikes
    smoothed_data = np.copy(trace.data)
    #smoothed_data[1:-1][spikes] = (trace.data[:-2][spikes] + trace.data[2:][spikes]) / 2
    smoothed_data[spikes] = trace_std

    # Update the trace data with the smoothed data
    trace.data = smoothed_data

def despike_trace(trace):
    # Parameters for despiking
    window_size = 5  # Size of the sliding window
    threshold_factor = 3  # Threshold factor to define spikes

    # Loop over the data and apply despiking
    despiked_data = trace.data.copy()
    for i in range(window_size, len(trace.data) - window_size):
        # Define the window of data around each point
        window = trace.data[i - window_size:i + window_size + 1]
        
        # Calculate local median and standard deviation
        local_median = np.median(window)
        local_std = np.std(window)
        
        # Identify spikes: data point deviates too much from the local median
        if np.abs(trace.data[i] - local_median) > threshold_factor * local_std:
            despiked_data[i] = local_median  # Replace with the local median

    # Update the trace data with the despiked data
    trace.data = despiked_data

Linux


In [2]:
sdsclient = Client(paths['SDS_TOP'])
thisSDSobj = SDS.SDSobj(paths['SDS_TOP'])

paths['PSI'] = os.path.join(paths['DROPBOX_DATA_TOP'], 'WellData', '05_DAILY_PSI')
paths['WLM'] = os.path.join(paths['DROPBOX_DATA_TOP'], 'WellData', '05_DAILY_WLM')
paths['EVENTS'] = os.path.join(paths['DROPBOX_DATA_TOP'], 'EVENTS')

paths['WWW_TOP'] = '/var/www/html/usfseismiclab.org/html/rocketcat'
paths['WWW_EVENTS']=os.path.join(paths['WWW_TOP'], 'EVENTS')
#paths['WWW_CONTINUOUS'] = os.path.join(paths['WWW_TOP'], 'CONTINUOUS')
paths['csv_launches'] = os.path.join(paths['WWW_TOP'], 'launches.csv')

launchesDF = LLE.removed_unnamed_columns(pd.read_csv(paths['csv_launches'], index_col=None, parse_dates=['datetime']))
#launchesDF['datetime']= pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])
preseconds=120
eventseconds=120
postseconds=120
taperseconds=600
aircolumns = ['AirPressureShallow_corrected', 'AirPressureDeep_corrected', '1226420_corrected', '1226429_corrected']
watercolumnsShallow = ['1226421_wlm', '1226419_wlm', '1226423_wlm']
watercolumnsIntermediate = ['2149882_wlm', '2151691_wlm', '2151692_wlm']
launchesDF['ontime']=None
launchesDF['offtime']=None
launchesDF['signalStrength']=None
launchesDF['Baro_amp']=None # units Pa
launchesDF['Baro_diff']=None # units Pa
launchesDF['6S_amp']=None # units mH20
launchesDF['6I_amp']=None # units mH20
launchesDF['6S_diff']=None #units mH20
launchesDF['6I_diff']=None #units mH20
launchesDF['6S_Pa']=None # units Pa
launchesDF['6I_Pa']=None # units Pa

# need to add a detector for sonic booms too!

In [3]:
inv1 = USF_instrument_responses.make_inv('FL', 'S39A1', '00', ['HHZ', 'HHN', 'HHE'])
inv2 = USF_instrument_responses.make_inv('FL', 'S39A2', '00', ['HHZ', 'HHN', 'HHE'])
inv3 = USF_instrument_responses.make_inv('FL', 'S39A3', '00', ['HHZ', 'HHN', 'HHE'])
inv4 = USF_instrument_responses.make_inv('FL', 'BCHH3', '00', ['HHZ', 'HHN', 'HHE'])
inv5 = USF_instrument_responses.make_inv('FL', 'BCHH4', '10', ['HHZ', 'HHN', 'HHE'])
inv = inv1 + inv2 + inv3 + inv4 + inv5
print(inv)


['Nanometrics', 'Centaur', '40 Vpp (1)', 'Off', 'Linear phase', '100']
['Nanometrics', 'Trillium Compact 120 (Vault, Posthole, OBS)', '754 V/m/s']
Channel Response
	From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 3.0172e+08 defined at 1.000 Hz
	6 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 754.3
		Stage 2: PolesZerosResponseStage from V to V, gain: 1
		Stage 3: CoefficientsTypeResponseStage from V to COUNTS, gain: 400000
		Stage 4: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
		Stage 5: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
		Stage 6: CoefficientsTypeResponseStage from COUNTS to COUNTS, gain: 1
Instrument Sensitivity:
	Value: 301720003.7617996
	Frequency: 1.0
	Input units: M/S
	Input units description: Velocity in Meters per Second
	Output units: COUNTS
	Output units description: Digital Counts

['Nanometrics', 'Centaur', '40 Vpp (1)', 'Off', 'Linear phase', '100']
['Nanometrics', 'Tril

In [ ]:
def remove_response(seismic_st, output='VEL'):
    this_st = seismic_st.copy()
    try:
        this_st.remove_response(inventory=inv, output=output, pre_filt=None) #correct to velocity
    except:
        for tr in this_st:
            print(tr)
            try:
                tr.remove_response(inventory=inv, output=output, pre_filt=None) #correct to displacement
            except Exception as e:
                print(e)
                this_st.remove(tr)    
    return this_st

for idx, eventrow in launchesDF.iterrows():
    # get start and end time. remember that spreadsheet is in UTC. well data are in local time but summer time is assumed throughout (UTC-4)
    launchtimeUTC = eventrow['datetime']

    # create event directory
    EVENTDIR = os.path.join(paths['WWW_EVENTS'], launchtimeUTC.isoformat())
    if not os.path.isdir(EVENTDIR):
        os.makedirs(EVENTDIR)  

    print('Processing launch at %s' % launchtimeUTC.strftime('%Y-%m-%d %H:%M:%S')) 
    st = sds2eventStream(launchtimeUTC, sdsclient, thisSDSobj, pretrig=preseconds+taperseconds, posttrig=eventseconds+postseconds+taperseconds)


    # Define the station you want to remove (e.g., 'STATION_NAME')
    station_to_remove = "DWPF"
    #st = st.select(lambda tr: tr.stats.station != station_to_remove)
    for tr in st:
        if tr.stats.station == station_to_remove:
            st.remove(tr)
        elif tr.data.size == 0 or all(tr.data == 0):
            st.remove(tr)


    st = st.select(network='FL') # while AM, XA, and 1R are also valid, we need to get instrument response data for them

    #st = st.filter(lambda trace: trace.data.size > 0 and not all(trace.data == 0))

    # all these functions safe for well traces too - if they were in the Stream objects as they were when we converted well data to SDS
    st.plot(equal_scale=False, outfile=os.path.join(EVENTDIR, 'raw.png'));
    clean(st, taperseconds) # despike, detrend, taper, high pass filter, trim
    infrasound_st = st.select(channel='HD?')
    seismic_st = st.select(channel='HH?')
    apply_calibration_correction(infrasound_st) 
  
    seismic_st = seismic_st.select(network='FL')
    velocity_st = remove_response(seismic_st, output='VEL')
    displacement_st = remove_response(seismic_st, output='DISP')
      
    # write corrected event out
    #correctedfile =  os.path.join(EVENTDIR, '%s.pkl' % launchtimeUTC.strftime('%Y%m%dT%H%M%S'))
    #st.write(correctedfile, format='PICKLE') 

    # SCAFFOLD
    # generate seismic and infrasound figures - but change to add RSAM pandas plots, spectrograms and spectra, etc.
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a custom figure size (width, height)
    infrasound_st.plot(fig=fig, equal_scale=False, outfile=os.path.join(EVENTDIR, 'infrasound.png'));
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a custom figure size (width, height)
    velocity_st.plot(fig=fig, equal_scale=False, outfile=os.path.join(EVENTDIR, 'velocity.png'));
    fig, ax = plt.subplots(figsize=(10, 6))  # Set a custom figure size (width, height)
    displacement_st.plot(fig=fig, equal_scale=False, outfile=os.path.join(EVENTDIR, 'displacement.png'));    
    # Optionally, set the aspect ratio (e.g., 2:1 aspect ratio)
    #ax.set_aspect(2.0)

    # now handle well data
    stime = launchtimeUTC - pd.Timedelta(seconds=preseconds) - pd.Timedelta(hours=4)
    etime = launchtimeUTC + pd.Timedelta(seconds=eventseconds+postseconds) - pd.Timedelta(hours=4)
    welldayfile0 = None
    if stime.day != eventrow['datetime'].day:
        welldayfile0 = os.path.join(paths['WLM'],stime.strftime('%Y%m%d.pkl'))
        if not os.path.isfile(welldayfile0):
            welldayfile0 = None
    if welldayfile0:
        welldaydf0 = pd.read_pickle(welldayfile0)
    else:
        welldaydf0 = None
    welldayfile = os.path.join(paths['WLM'],launchtimeUTC.strftime('%Y%m%d.pkl'))
    if os.path.isfile(welldayfile):
        welldaydf = pd.read_pickle(welldayfile)
        if isinstance(welldaydf0, pd.DataFrame):
            welldaydf = pd.concat([welldaydf0, welldaydf])
        #welldaydf = welldaydf.filter(regex='^(?!.*wldiff$)')
        print(f'Subsetting {os.path.basename(welldayfile)} from {stime} to {etime}')
        welldaydf['datetime'] = pd.to_datetime(welldaydf['datetime'])
        mask = (welldaydf['datetime']>=stime) & (welldaydf['datetime']<=etime)
        welldaydf = welldaydf.loc[mask]
        welldaydf = welldaydf.dropna(subset=['datetime'])

        if len(welldaydf)>0:
            channelsdf = pd.DataFrame(columns=['nslc', 'amplitude', 'DC_before', 'DC_after', 'DC_diff', 'units', 'corr_coef', 'shift'])

            # fix units
            for col in welldaydf.columns:
                if col.endswith('wlm'):
                #if col in watercolumnsShallow + watercolumnsIntermediate:
                    welldaydf[col] = welldaydf[col]
                elif col in aircolumns:
                    welldaydf[col] = welldaydf[col] * 6894.76 # PSI to Pascals
            
            ax1 = welldaydf.plot(x='datetime', y=intersect(welldaydf, aircolumns), ylabel='Air Pressure (PSI)', title=f'Air Pressure for event at {launchtimeUTC}')
            plt.savefig(os.path.join(EVENTDIR, 'dataframe_aircolumns.png'))
            ax2 = welldaydf.plot(x='datetime', y=intersect(welldaydf, watercolumnsShallow), ylabel='NAVD88 (m)', title=f'Water Level in Shallow well for event at {launchtimeUTC}')
            plt.savefig(os.path.join(EVENTDIR, 'dataframe_watercolumns6S.png'))
            ax3 = welldaydf.plot(x='datetime', y=intersect(welldaydf, watercolumnsIntermediate), ylabel='NAVD88 (m)', title=f'Water Level in Intermediate well for event at {launchtimeUTC}')
            plt.savefig(os.path.join(EVENTDIR, 'dataframe_watercolumns6I.png'))
            
            welldaydf.to_pickle(os.path.join(EVENTDIR, 'welldata.pkl'))

            st = dataframe2stream(welldaydf)
            best_trig = detectEvent(st)
            pad_secs = 10
            if len(best_trig)>0:    

                st3 = st.copy()
                st3.trim(starttime=best_trig['time']-pad_secs, endtime=best_trig['time']+pad_secs+best_trig['duration'])
                st3.write(os.path.join(EVENTDIR, 'welldata.mseed'), format='MSEED')
                AP6S = st3.select(id='6S.Baro..HDD')
                bool_xcorr = False
                if len(AP6S)==1:
                    bool_xcorr = True
                    AP6S = AP6S.copy()[0].detrend()
                for channel in ['HDD', 'WLM']:
                    if channel=='HDD':
                        units = 'Pa'
                        titlestr = f'Air Pressure (Pa) for Event {launchtimeUTC}'
                    elif channel=='WLM':
                        units = 'm'
                        titlestr = f'Water Level (m) for Event {launchtimeUTC}'    
                    st4 = st3.select(channel=channel)

                    # time series plots on raw Stream
                    st4.plot(equal_scale=True, outfile=os.path.join(EVENTDIR, f'stream_{channel}.png'));


                    # we do the spectral processing on a detrended Stream
                    st5 = st4.copy()
                    st5.detrend('linear')
                    spobj = Spectrograms.icewebSpectrogram(stream=st5)
                    spobj = spobj.precompute()
                    spobj.plot(fmin=0.1, fmax=50.0, equal_scale=True, title=titlestr, outfile=os.path.join(EVENTDIR, f'sgram_{channel}.png'))
                    #st3.spectrogram()
                    spobj.compute_amplitude_spectrum(compute_bandwidth=True)
                    spobj.plot_amplitude_spectrum(normalize=True, title=titlestr)  
                    plt.savefig(os.path.join(EVENTDIR, f'spectrum_{channel}.png'))
                
                    for i,tr in enumerate(st5):
                        if bool_xcorr:
                            correlation = correlate(tr, AP6S, 100)
                            bestshift, bestvalue = xcorr_max(correlation, abs_max=True)
                        tr_before = st4[i].copy()
                        tr_before = tr_before.trim(starttime=best_trig['time']-pad_secs, endtime=best_trig['time']-pad_secs/2)
                        tr_after = st4[i].copy()
                        tr_after = tr_after.trim(starttime=best_trig['time']+best_trig['duration']+pad_secs/2, endtime=best_trig['time']+best_trig['duration']+pad_secs)
                        DC_before=np.median(tr_before.data)
                        DC_after=np.median(tr_after.data)
                        new_row = {'nslc':tr.id, 'amplitude':tr.max(), 'DC_before':DC_before, 'DC_after':DC_after, 'DC_diff':DC_after-DC_before, 'units':units}
                        #nErow['bw_min']=
                        #new_row['bw_max']=
                        new_row['corr_coef']=bestvalue
                        new_row['shift']=bestshift
                        channelsdf.loc[len(channelsdf)] = new_row
                display(channelsdf)
                ''' what statistics can we add to the dataframe for each event?
                * maximum absolute amplitude (from mean)
                * bandwidth?
                * detection on and off time
                * correlations between each trace - and a median correlation?
                * correlation best time lag between each trace 
                * DC level before and after
                * summary or event level (or per station) stats can go in the event catalog, more detailed per channel stats can go in the event
                '''

            if len(channelsdf)>0:
                # save channelsdf
                channelsdf.to_csv(os.path.join(EVENTDIR, 'channelsDF.csv'), index=False)

                # add content to launchesDF for this event
                airdf = channelsdf[channelsdf['nslc'].str.contains('.Baro.')]
                waterdf = channelsdf[channelsdf['nslc'].str.endswith('WLM')]
                water6S = waterdf[waterdf['nslc'].str.startswith('6S')]
                water6I = waterdf[waterdf['nslc'].str.startswith('6I')]
                launchesDF.at[idx, 'signalStrength'] = np.sum(best_trig['cft_peaks'])
                launchesDF.at[idx, 'Baro_amp'] = airdf['amplitude'].abs().mean(axis=0)
                launchesDF.at[idx,'Baro_diff'] = airdf['DC_diff'].mean(axis=0)
                launchesDF.at[idx, '6S_amp'] = water6S['amplitude'].abs().mean(axis=0)
                launchesDF.at[idx, '6I_amp'] = water6I['amplitude'].abs().mean(axis=0)
                launchesDF.at[idx, '6S_diff'] = water6S['DC_diff'].mean(axis=0)
                launchesDF.at[idx, '6I_diff'] = water6I['DC_diff'].mean(axis=0)
                
# remove any blank columns in launchesDF
launchesDF = launchesDF.dropna(axis=1, how='all')
if '6S_Pa' in launchesDF: # we have well data
    display(launchesDF.columns)
    launchesDF['6S_Pa'] = launchesDF['6S_amp']*9806.65
    launchesDF['6I_Pa'] = launchesDF['6I_amp']*9806.65
    launchesDF.to_csv(os.path.join(paths['WWW_EVENTS'], 'launchesDF.csv'))
    launchesDF.plot(x='datetime',y=['Baro_amp', '6S_Pa', '6I_Pa'], style='o', ylabel='Pa', title='Comparing pressures in air, and in each well')
    plt.savefig(os.path.join(paths['WWW_EVENTS'], 'catalog_pascals.png'))
    display(launchesDF[['datetime', 'Baro_amp', '6S_Pa', '6I_Pa']])

Processing launch at 2022-04-01 16:24:16
st = read("/data/SDS/2022/FL/BCHH2/HD4.D/FL.BCHH2.10.HD4.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH2/HD5.D/FL.BCHH2.10.HD5.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH2/HD6.D/FL.BCHH2.10.HD6.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH2/HD7.D/FL.BCHH2.10.HD7.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH2/HD8.D/FL.BCHH2.10.HD8.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH2/HD9.D/FL.BCHH2.10.HD9.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH3/HHE.D/FL.BCHH3.00.HHE.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH3/HHN.D/FL.BCHH3.00.HHN.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH3/HHZ.D/FL.BCHH3.00.HHZ.D.2022.091")
st = read("/data/SDS/2022/FL/BCHH3/HDF.D/FL.BCHH3.10.HDF.D.2022.091")
st = read("/data/SDS/2022/FL/S39A1/HHE.D/FL.S39A1.00.HHE.D.2022.091")
st = read("/data/SDS/2022/FL/S39A1/HHN.D/FL.S39A1.00.HHN.D.2022.091")
st = read("/data/SDS/2022/FL/S39A1/HHZ.D/FL.S39A1.00.HHZ.D.2022.091")
st = read("/data/SDS/2022/FL/S39A1/HDF.D/FL.S39A1